# 模仿《安娜卡列尼娜》的文本生成代码，构建一个平凡的世界的生成模型

In [1]:
import time
from collections import namedtuple
import numpy as np
import pandas as pd
import jieba
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Instructions for updating:
non-resource variables are not supported in the long term


## 1 数据加载与预处理

In [ ]:
# with open ('./mini-data.txt') as f:
#     text=f.read()
# #     改变分词处理,不是分词,而是分句子

# # text = text.replace('，', ' ').replace('。', ' ').replace('？', ' ').replace('：', ' ').replace('\n',' ').replace('！',' ')
# # token_text = tf.keras.preprocessing.text.text_to_word_sequence(text, split=' ')
# text=text.split('\n\n')
# vocab=[]
# # 对文本进行jieba分词处理
# for sentence in text:
#     seg=jieba.lcut(sentence)
#     vocab.append(seg)
# #     降维
# all_vocab=sum(vocab,[])
# vocab=set(all_vocab)

In [ ]:
# vocab_to_int={c:i for i,c in enumerate(vocab)}
# int_to_vocab=dict(enumerate(vocab))
# # 将分词后的文本转化为整数
# encoded=np.array([vocab_to_int[c] for c in all_vocab],dtype=np.int32)

In [ ]:
# a=','
# a in vocab_set
# encoded

In [2]:
with open ('./mini-data.txt') as f:
    text1=f.read()
vocab = set(text1)
vocab_to_int = {c:i for i,c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text1],dtype=np.int32)

In [3]:
vocab

{'说',
 '提',
 '毫',
 '焦',
 '害',
 '诗',
 '姿',
 '责',
 '庄',
 '把',
 '迷',
 '娃',
 '展',
 '蔑',
 '业',
 '排',
 '撑',
 '度',
 '迫',
 '最',
 '拳',
 '梅',
 '关',
 '续',
 '味',
 '赶',
 '杂',
 '扬',
 '笔',
 '遥',
 '打',
 '艰',
 '烩',
 '7',
 '简',
 '存',
 '气',
 '候',
 '所',
 '从',
 '见',
 '碌',
 '滑',
 '果',
 '绵',
 '免',
 '遭',
 '估',
 '搀',
 '起',
 '扒',
 '副',
 '短',
 '来',
 '沿',
 '概',
 '娘',
 '找',
 '户',
 '檐',
 '缩',
 '凳',
 '洞',
 '熟',
 '架',
 '它',
 '垒',
 '晓',
 '离',
 '胶',
 '角',
 '怨',
 '宠',
 '舍',
 '动',
 '窃',
 '猜',
 '瓷',
 '郁',
 '抢',
 '成',
 '粗',
 '矩',
 '怯',
 '瘫',
 '稼',
 '明',
 '奢',
 '无',
 '裕',
 '蓝',
 '强',
 '苦',
 '浪',
 '大',
 '麦',
 '婆',
 '愉',
 '迎',
 '拾',
 '伙',
 '凡',
 '象',
 '经',
 '取',
 '前',
 '昏',
 '披',
 '泛',
 '勤',
 '”',
 '假',
 '年',
 '助',
 '有',
 '碗',
 '美',
 '破',
 '漾',
 '嘈',
 '意',
 '侈',
 '着',
 '支',
 '约',
 '溃',
 '厨',
 '书',
 '惯',
 '蹒',
 '惹',
 '深',
 '能',
 '装',
 '激',
 '病',
 '嘶',
 '没',
 '傻',
 '革',
 '担',
 '偶',
 '霞',
 '望',
 '帮',
 '劲',
 '境',
 '汗',
 '尤',
 '巷',
 '）',
 '恋',
 '搜',
 '爱',
 '躲',
 '五',
 '许',
 '那',
 '嚼',
 '宁',
 '薄',
 '满',
 '应',
 '严',
 '跚',
 '含',
 '揩'

## 2 数据集划分

#### 训练过程,输入的数据是以batch形式完成的,每个batch包含batch_size个样本,每个样本长度设置为n_seqs,即包含n_seqs个字符,现在有一个长度为N个字符的原始文本,那么训练集可划分为m个batch

$$ m=N/(batch\_size*n\_seqs)$$

#### 我们定义一个batch中的序列个数为N（batch_size），定义单个序列长度为M（也就是我们的steps）。那么实际上我们每个batch是一个N x M的数组。在这里我们重新定义batch_size为一个N x M的数组，而不是batch中序列的个数。在上图中，当我们设置N=2， M=3时，我们可以得到每个batch的大小为2 x 3 = 6个字符，整个序列可以被分割成12 / 6 = 2个batch。



In [4]:
# 因为LSTM是用前一个字符预测后一个字符,所以label的值为x后移一位
def get_batches(arr,n_seqs,n_steps):
    '''
    对已有的数组进行mini-batch划分
    arr:带划分的数组
    n_seqs:一个batch中的序列个数
    n_steps:单个序列包含的字符数
    '''
    batch_size=n_seqs * n_steps
    n_batches=int(len(arr) / batch_size)
#     保留完整的batch
    arr=arr[:batch_size * n_batches]
    arr=arr.reshape(n_seqs,-1)
    
    for n in range(0,arr.shape[1],n_steps):
    # inputs
        x = arr[:, n:n+n_steps]
        # targets
        y = np.zeros_like(x)
#         y为当前batch中向左平移一个单位的结果
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y  

In [5]:
batches=get_batches(encoded,10,150)
x,y=next(batches)

In [6]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])
print(x.shape)
print(y.shape)

x
 [[ 612  718 1122  288  288  240  240  240  240  512]
 [ 341  492 1357  981 1214 1184  491 1077  660 1148]
 [1337  917  477  290  699  158 1012 1337  574  593]
 [ 481 1201  266  103 1148  743  357  718 1269  416]
 [ 532  131 1306 1005  796  114  718 1184    0  492]
 [ 481 1357 1049  603  803  472  628  406  459  971]
 [ 354  366 1268  151  153 1012 1064  481 1201   88]
 [ 971 1290  687  412 1271  929  108  641 1341  461]
 [ 752  956  387  971  451 1303  491 1077  380   43]
 [  62 1077  752  359    0  481  918  721  593  462]]

y
 [[ 718 1122  288  288  240  240  240  240  512  590]
 [ 492 1357  981 1214 1184  491 1077  660 1148  242]
 [ 917  477  290  699  158 1012 1337  574  593  122]
 [1201  266  103 1148  743  357  718 1269  416 1024]
 [ 131 1306 1005  796  114  718 1184    0  492  641]
 [1357 1049  603  803  472  628  406  459  971 1226]
 [ 366 1268  151  153 1012 1064  481 1201   88  449]
 [1290  687  412 1271  929  108  641 1341  461  971]
 [ 956  387  971  451 1303  491 1077  

## 3 模型构建
####  模型构建部分主要包括了输入层、LSTM层、输出层、loss、optimize等部分的构建

### 3.1 输入层
每次输入一个batch

In [7]:
def build_inputs(num_seqs,num_steps):
    '''
    构建输入层
    
    num_seqs:每个batch中的序列个数
    num_steps:每个序列包含的字符数
    '''
    inputs=tf.placeholder(tf.int32,shape=(num_seqs,num_steps),name='inputs')
    targets=tf.placeholder(tf.int32,shape=(num_seqs,num_steps),name='targets')  
    
#     加入keep_prob：dropout中保留的概率
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    
    return inputs,targets,keep_prob    

### 3.2 LSTM层

tf.contrib.rnn有两个包：

BasicLSTMCell: 平常说的LSTM.

LSTMCell: LSTM升级版，加了clipping，projection layer,peep-hole等操作。

MultiRNNCell:实现了对基本LSTM cell的顺序堆叠。

dynamic_rnn:实现循环调用LSTMCell，实现神经网络前向计算。

In [8]:
def build_lstm(lstm_size,num_layers,batch_size,keep_prob):
    '''
    构建lstm层
    
    keep_prob:dropout保留的概率
    num_layers:lstm的隐藏层的数目
    batch_size
    '''
    def get_a_cell(lstm_size,keep_prob):
        lstm=tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
        drop=tf.nn.rnn_cell.DropoutWrapper(lstm,output_keep_prob=keep_prob)
        return drop
    
# #     构建一个基本的lstm单元
#     lstm=tf.nn.rnn_cell.BasicLSTMCell(lstm_size) 
    
# #     添加dropout
#     drop=tf.nn.rnn_cell.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    
#     堆叠:构建多隐层神经网络
# 使用RNN堆叠函数将前面构造的lstm_cell多层堆叠得到cell，堆叠次数为lstm中隐层数目-num_layers
    cell=tf.nn.rnn_cell.MultiRNNCell([get_a_cell(lstm_size,keep_prob) for _ in range(num_layers)])
#     设置LSTM单元的初始化状态为0
    initial_state=cell.zero_state(batch_size,tf.float32)
    
    return cell,initial_state

### 3.3 输出层
指整个网络的最终输出

输出层采用softmax,它与LSTM进行全连接,对于每一个字符来说,它经过LSTM后的输出大小为(1,lstm\_size),所以一个batch经过LSTM输出后的大小为(batch\_size\*n\_seqs,lstm\_size).要将这个输出与softmax全连接层建立连接,就需要将LSTM的输出reshape为(batch\_size\*n\_seqs,lstm\_size)

softmax层的结点数应该是vocab的大小(要计算概率分布).因此整个LSTM层到softmax层权重矩阵的大小为(lstm\_size,vocab\_size)

最终的输出logits为(batch\_size\*n\_seqs,vocab\_size)

In [14]:
def build_output(lstm_output,in_size,out_size):
    '''
    构造输出层
    
    lstm_output:lstm层的输出结果
    in_size:lstm输出层重塑后的size
    out_size:softmax层的size
    '''
    
    seq_output=tf.concat(lstm_output,1)
    x=tf.reshape(seq_output,[-1,in_size])
    
    # 将lstm层与softmax层全连接
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal([in_size, out_size], stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # 计算logits
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # softmax层返回概率分布
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### 3.4 训练误差计算
采用tf.nn.softmax_cross_entropy_with_logits交叉熵来计算loss。

该函数进行两步运算：

首先对logits进行softmax计算，根据softmax计算后的结果和labels来计算交叉熵损失。

计算出的结果是向量形式, shape = (batch_size,)，因此需要 reduce_mean来进行求均值。

In [9]:
def build_loss(logits, targets, lstm_size, num_classes):
    '''
    根据logits和targets计算损失
    
    logits: 全连接层的输出结果（不经过softmax）
    targets: targets
    lstm_size
    num_classes: vocab_size
        
    '''
    
    # One-hot编码
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss 

### 3.5 Optimizer
采用gradient clippling的方式来防止梯度爆炸。即通过设置一个阈值，当gradients超过这个阈值时，就将它重置为阈值大小，这就保证了梯度不会变得很大。
优化器的构造流程：  
1.找到网络中的可训练参数，因为要对 w,b进行更新，  
2.计算梯度：tf.gradients(loss,训练参数)，  
3.梯度裁剪：tf.clip_by_global_norm(梯度，阈值)，  
4.实例化一个优化器：train_op = tf.train.AdadeltaOptimizer(学习率)  
5.优化器进行梯度下降更新训练参数，得到一个op：optimizer = train_op.apply_gradients(zip(grades,tvars))

In [10]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' 
    构造Optimizer
   
    loss: 损失
    learning_rate: 学习率
    
    '''
    
    # 使用clipping gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### 3.6 模型组合
使用tf.nn.dynamic_run来运行RNN序列

In [11]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # 如果sampling是True，则采用SGD
#         预测阶段:batch_size=1,文本长度=1,即输入一个字符,预测下一个字符
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # 输入层
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # LSTM层
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        # 对输入进行one-hot编码
        x_one_hot = tf.one_hot(self.inputs, num_classes)
#         print('$$$$$$$$$$',x_one_hot)
        # 运行RNN
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # 预测结果
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss 和 optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## 4 模型训练
参数设置
在模型训练之前，我们首先初始化一些参数，我们的参数主要有：

num_seqs: 单个batch中序列的个数

num_steps: 单个序列中字符数目

lstm_size: 隐层结点个数

num_layers: LSTM层个数

learning_rate: 学习率

keep_prob: dropout层中保留结点比例

In [12]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability
config = tf.ConfigProto(
        allow_soft_placement=True, # 自动选择CPU 还是GPU
        log_device_placement=False, # 是否打印设备日志
    )
epochs = 500
# 每n轮进行一次变量保存
save_every_n = 200

In [15]:
model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
#             print('counter:',counter)
#             print('@@@@@@@@@@@')
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state
                   }
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            # control the print lines
            if counter % 50 == 0:
                print('轮数: {}/{}... '.format(e+1, epochs),
                      '训练步数: {}... '.format(counter),
                      '训练误差: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))

            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

轮数: 50/500...  训练步数: 50...  训练误差: 5.8208...  0.4840 sec/batch
轮数: 100/500...  训练步数: 100...  训练误差: 5.2414...  0.4822 sec/batch
轮数: 150/500...  训练步数: 150...  训练误差: 3.6093...  0.4874 sec/batch
轮数: 200/500...  训练步数: 200...  训练误差: 2.3177...  0.4829 sec/batch
轮数: 250/500...  训练步数: 250...  训练误差: 1.7238...  0.4877 sec/batch
轮数: 300/500...  训练步数: 300...  训练误差: 1.3726...  0.4843 sec/batch
轮数: 350/500...  训练步数: 350...  训练误差: 1.1274...  0.4874 sec/batch
轮数: 400/500...  训练步数: 400...  训练误差: 0.9327...  0.4915 sec/batch
轮数: 450/500...  训练步数: 450...  训练误差: 0.7626...  0.4892 sec/batch
轮数: 500/500...  训练步数: 500...  训练误差: 0.6455...  0.4824 sec/batch


In [17]:
# 查看checkpoints
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i500_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i500_l512.ckpt"

## 5 文本生成
现在我们可以基于我们的训练参数进行文本的生成。当我们输入一个字符时，LSTM会预测下一个字符，我们再将新的字符进行输入，这样能不断的循环下去生成本文。

为了减少噪音，每次的预测值我会选择最可能的前5个进行随机选择，比如输入h，预测结果概率最大的前五个为[o,e,i,u,b]，我们将随机从这五个中挑选一个作为新的字符，让过程加入随机因素会减少一些噪音的生成。

In [18]:
def pick_top_n(preds, vocab_size, top_n=5):
    """
    从预测结果中选取前top_n个最可能的字符
    
    preds: 预测结果
    vocab_size
    top_n
    """
    p = np.squeeze(preds)
    # 将除了top_n个预测值的位置都置为0
    p[np.argsort(p)[:-top_n]] = 0
    # 归一化概率
    p = p / np.sum(p)
    # 随机选取一个字符
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

np.squeeze(),从数组的形状中删除单维条目，即把shape中为1的维度去掉。  
在预测阶段，输入样本形状为(1,1)  
preds为输出层的输出,即（1，83），表示当前输入字符为字符表中每个字符的概率。  
p = np.array([长度为83的列表])，取top_n个值。

In [19]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="同村"):
    """
    生成新文本
    
    checkpoint: 某一轮迭代的参数文件
    n_sample: 新闻本的字符长度
    lstm_size: 隐层结点数
    vocab_size
    prime: 起始文本
    """
    # 将输入的单词转换为单个字符组成的list
    samples = [c for c in prime]
#     samples=prime
    # sampling=True意味着batch的size=1 x 1
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # 加载模型参数，恢复训练
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            # 输入单个字符
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state
                   }
            preds, new_state = sess.run([model.prediction, model.final_state],
                                        feed_dict=feed)
            
#         x = np.zeros((1, 1))
#         # 输入单个字符
#         print('fg', vocab_to_int[samples])
#         x[0,0] = vocab_to_int[samples]
#         feed = {model.inputs: x,
#                 model.keep_prob: 1.,
#                 model.initial_state: new_state}
#         preds, new_state = sess.run([model.prediction, model.final_state], 
#                                      feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        # 添加字符到samples中
#         samples=samples+int_to_vocab[c]
        samples.append(int_to_vocab[c])
        
        # 不断生成字符，直到达到指定数目
        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
#             samples=samples+int_to_vocab[c]
        
    return ''.join(samples)

在for循环中，每次输出的结果包含字符和隐状态，隐状态作为下一步网络的输入，字符保存到列表作为最后的生成文本。

tf.train_latest_checkpoint()方法，可以选择最后的训练的参数作为网络参数。

In [20]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i500_l512.ckpt'

In [21]:
# 选用最终的训练参数作为输入进行文本生成
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="少平知道")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i500_l512.ckpt
少平知道，部要地人有发

    一一身感日。他在山蒙的地是一点点，他得得一瘦子。大年年一的菜，其且起经最得往一有的子而而膊腕敏的一来都。有得些的人陷，在在种的的的光子都有等中力而着地要十钱菜的，其也刚高体的裳，还以算得是人人，好有她的行。这样饭饭。可一也到到不，少是前跛？说除这几下日日日，她她的拿个着白馍只只只到了里里听着着，剩的的两两个粱面，看在的上课。课他来来，上了地…地…去。

    他年个下子，这不走地生了，他家家人里有着一自。说这：的小菜，也没是些山或馍—，有年点力力地动—为为去，谁里刚有几个的……”每这个样、的尽，他也不说到了，”

     在你的动动没我我下下。我上爸个的家。我爷爷道事的她，这说个有话””你你着你我还我爸爸我…



    “你。我爸子没爷过。”我 我我爸爸是过面馍的。

    “你？？有到爸了？”

     “嗯。”

    “你爸在？？””
   ““能”。

    ““你？””我我爸上看””

    ““。”””有过爸看过””

    “你能你，”我”爷

    “你你你…””我

    ““爸能””””

     ““能”””我我在爸上过。

    ““不。我我我来地了。

    “能爸两个。要爷过地，”我平以爸后，是有过”。
   “你过，就没有爷梅。她 
   “过爸借？他过…””

    “你爸我过，”爷爷…

    “你上我我我我过？”
   ““爸爸。爷过了。

    ““爸？？有过过”。

    ““爸。我我…过…”

    “爸爸过？？我

    “你。”我没有爸”。    “能能。我爸一是看。”

    “你爸？”我过…”

     “爸爸。”爷过过。
   “爸爸过岩？？”

    “你嗯。我

    “你爸？？…”少平然有有点点。这惊。
   班的也又到我他，要了打。。
   少平后，一没没在地的。。他整这感，只不不出出了活。是里就他，她起了行行。。
过、、、，菜，也用不到打了。他里前下点”有她然感穿的人都不了。
    各忙雪场场心来后的。他他然感，只在天天黑地家，不了他寒。。里那天地过，在地蒙的子子子还看着惊点点雪而头头